In [4]:
!python -V

Python 3.13.6


In [5]:
import pandas as pd

In [6]:
import pickle

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [10]:
import mlflow


mlflow.set_tracking_uri("file:///E:/MLops/LearningMLops/01-intro/myenv/mlruns")
mlflow.set_experiment("nyc-taxi-experiment")

E:\MLops\LearningMLops\mlops-env\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/26 11:15:58 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///E:/MLops/LearningMLops/01-intro/myenv/mlruns/805662149743723148', creation_time=1764135958542, experiment_id='805662149743723148', last_update_time=1764135958542, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [11]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [12]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-03.parquet')

In [13]:
len(df_train), len(df_val)

(51497, 55139)

In [14]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [15]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [16]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)

35.380256321466476

In [18]:
import os

# Create the directory if it doesn't exist
os.makedirs("models", exist_ok=True)

# Now save your file
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)


In [19]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [20]:
from mlflow.tracking import MlflowClient
client = MlflowClient()
runs = client.search_runs(experiment_ids=["0"])  # "0" is the default experiment
for run in runs:
    print(f"Run ID: {run.info.run_id}, RMSE: {run.data.metrics.get('rmse')}")


In [21]:
!which python


'which' is not recognized as an internal or external command,
operable program or batch file.


In [22]:
import xgboost as xgb

In [23]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

E:\MLops\LearningMLops\mlops-env\Lib\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [24]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [25]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [26]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:8.52037                                                                                                                                     
[1]	validation-rmse:7.99931                                                                                                                                     
[2]	validation-rmse:7.56042                                                                                                                                     
[3]	validation-rmse:7.19118                                                                                                                                     
[4]	validation-rmse:6.88340                                                                                                                                     
  0%|                                                                                                                    | 0/50 [00:00<?, ?trial/s, best loss=?]

E:\MLops\LearningMLops\mlops-env\Lib\site-packages\xgboost\callback.py:386: UserWarning: [11:18:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[5]	validation-rmse:6.62865                                                                                                                                     
[6]	validation-rmse:6.41823                                                                                                                                     
[7]	validation-rmse:6.24439                                                                                                                                     
[8]	validation-rmse:6.10154                                                                                                                                     
[9]	validation-rmse:5.98390                                                                                                                                     
[10]	validation-rmse:5.88801                                                                                                                                    
[11]	validation-rmse:5.81045      

2025-11-26 11:18:26 ERROR [hyperopt.fmin] job exception: got an unexpected keyword argument 'squared'


  0%|                                                                                                                    | 0/50 [00:13<?, ?trial/s, best loss=?]


TypeError: got an unexpected keyword argument 'squared'

In [25]:
mlflow.xgboost.autolog(disable=True)

In [32]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

E:\MLops\LearningMLops\mlops-env\Lib\site-packages\xgboost\callback.py:386: UserWarning: [11:27:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:8.57577
[1]	validation-rmse:8.09018
[2]	validation-rmse:7.67105
[3]	validation-rmse:7.30866
[4]	validation-rmse:6.99858
[5]	validation-rmse:6.73334
[6]	validation-rmse:6.50917
[7]	validation-rmse:6.31710
[8]	validation-rmse:6.15563
[9]	validation-rmse:6.01783
[10]	validation-rmse:5.90384
[11]	validation-rmse:5.80630
[12]	validation-rmse:5.72466
[13]	validation-rmse:5.65568
[14]	validation-rmse:5.59799
[15]	validation-rmse:5.54921
[16]	validation-rmse:5.50784
[17]	validation-rmse:5.47255
[18]	validation-rmse:5.44452
[19]	validation-rmse:5.41848
[20]	validation-rmse:5.39584
[21]	validation-rmse:5.37596
[22]	validation-rmse:5.35988
[23]	validation-rmse:5.34551
[24]	validation-rmse:5.33313
[25]	validation-rmse:5.32320
[26]	validation-rmse:5.31272
[27]	validation-rmse:5.30378
[28]	validation-rmse:5.29622
[29]	validation-rmse:5.28884
[30]	validation-rmse:5.28277
[31]	validation-rmse:5.27772
[32]	validation-rmse:5.27383
[33]	validation-rmse:5.26969
[34]	validation-rmse:5.2

2025/11/26 11:28:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/26 11:28:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [30]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

FileNotFoundError: [WinError 2] The system cannot find the file specified